In [1]:
# Start by importing necessary packages
import requests
import pandas as pd
from IPython.display import display
from io import StringIO

from dstapi import DstApi # The helper class

**Alternative to using DstApi:**

In [2]:
# Directly embed parameters in the URL with response.get()
requests.get('https://api.statbank.dk/v1' + '/tableinfo' + "?id=NAN1&format=JSON").json()

# Pass a dictionary of parameters to requests.get()
params = {'id': 'NAN1', 'format': 'JSON'}
requests.get('https://api.statbank.dk/v1' + '/tableinfo', params=params).json()

# Use response.post() - note the change in the name of the parameter about the table's name
# I'm also adding here a language parameter - most tables are available in both Danish and English
params = {'table': 'NAN1', 'format': 'JSON', 'lang':'en'}
requests.post('https://api.statbank.dk/v1' + '/tableinfo', json=params).json()

{'id': 'NAN1',
 'text': 'Demand and supply',
 'description': 'Demand and supply by transaction, price unit and time',
 'unit': '-',
 'suppressedDataValue': '0',
 'updated': '2023-03-31T08:00:00',
 'active': True,
 'contacts': [{'name': 'Bo Siemsen',
   'phone': '39173069',
   'mail': 'bsm@dst.dk'}],
 'documentation': {'id': '0cf24458-15d1-486d-ad2d-8f4d16dd37ad',
  'url': 'https://www.dst.dk/documentationofstatistics/0cf24458-15d1-486d-ad2d-8f4d16dd37ad'},
 'footnote': {'text': 'Chained values do not show additivity. By chained values you cannot find totals by simple addition of components. You need to use a specific formula as described here - https://www.dst.dk/ext/national/NR-DOK.',
  'mandatory': True},
 'variables': [{'id': 'TRANSAKT',
   'text': 'transaction',
   'elimination': False,
   'time': False,
   'values': [{'id': 'B1GQK', 'text': 'B.1*g Gross domestic product'},
    {'id': 'P7K', 'text': 'P.7 Imports of goods and services'},
    {'id': 'P71K', 'text': 'P.71 Import of go

**Using DstApi (pip install git+https://github.com/alemartinello/dstapi):**

In [47]:
# Initialize the class with the target tables
NationalAccount = DstApi('NAN1')                # Unit : 2010-prices, chained values, bill DKK
EmploymentPopuplation = DstApi('NAHB')          # Unit : number of people
Capital = DstApi('NAHD34')                      # Unit : m DKK
Unemployment = DstApi('AULAAR')                 # Unit : % of labourforce

Income = DstApi('INDKP201')                     # Unit : DKK, fixed prices
Wealth = DstApi('FORMUE11')                     # Unit : DKK, fixed prices / Using 202-definiion of net-wealth

FirstYear = 2000                                # Define first year of data (in cases where prior data exists)

Table INDKP201: Main table for personal income statistics by type of income, sex, age, population, price unit, unit and time
Last update: 2022-11-24T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,INDKOMSTTYPE,39,100,1 Disposable income (2+30-31-32-35),290,Taxable income,False
1,KOEN,3,MOK,"Men and women, total",K,Women,False
2,ALDER,15,14TOT,"Total, 15 years and over",80-00,80 years and over,False
3,POPU,2,5000,All regardless of having the type of income,5020,Only people with the type of income,False
4,PRISENHED,2,005,Constant prices,006,Nominal prices,False
5,ENHED,5,0050,Mean (DKK),105,People (Number),False
6,Tid,26,1996,1996,2021,2021,True


## National account

In [4]:
display(NationalAccount.variable_levels('TRANSAKT', language='en'))

,id,text
0,B1GQK,B.1*g Gross domestic product
1,P7K,P.7 Imports of goods and services
2,P71K,P.71 Import of goods
3,P72K,P.72 Import of services
4,TFSPR,Supply
5,P6D,P.6 Exports of goods and services
6,P61D,P.61 Export of goods
7,P62D,P.62 Export of services
8,P31S1MD,P.31 Private consumption
9,P31S14D,P.31 Household consumption expenditure


**Define variables of interest:**

In [6]:
params = {
    'table': 'NAN1',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'TRANSAKT', 'values': ['B1GQK','P7K','P6D','P31S1MD','P3S13D','P5GD']},
        {'code': 'PRISENHED', 'values': ['LAN_M']},
        {'code': 'Tid', 'values': [f'>={FirstYear}<']}
        ]
    }
SupplyBalanceVar = NationalAccount.get_data(params=params).set_index(['TID']).sort_values(['TID'])

**Create dataframe:**

In [7]:
SupplyBalanceDF = pd.DataFrame()
SupplyBalanceDF['Y'] = SupplyBalanceVar[['INDHOLD']].loc[SupplyBalanceVar['TRANSAKT'] == 'B.1*g Gross domestic product'].copy()
SupplyBalanceDF['M'] = SupplyBalanceVar[['INDHOLD']].loc[SupplyBalanceVar['TRANSAKT'] == 'P.7 Imports of goods and services'].copy()
SupplyBalanceDF['X'] = SupplyBalanceVar[['INDHOLD']].loc[SupplyBalanceVar['TRANSAKT'] == 'P.6 Exports of goods and services'].copy()
SupplyBalanceDF['C'] = SupplyBalanceVar[['INDHOLD']].loc[SupplyBalanceVar['TRANSAKT'] == 'P.31 Private consumption'].copy()
SupplyBalanceDF['G'] = SupplyBalanceVar[['INDHOLD']].loc[SupplyBalanceVar['TRANSAKT'] == 'P.3 Government consumption expenditure'].copy()
SupplyBalanceDF['I'] = SupplyBalanceVar[['INDHOLD']].loc[SupplyBalanceVar['TRANSAKT'] == 'P.5g Gross capital formation'].copy()
SupplyBalanceDF = SupplyBalanceDF.apply(pd.to_numeric)
SupplyBalanceDF['M_ratio'] = SupplyBalanceDF['M']/SupplyBalanceDF['Y']
SupplyBalanceDF['X_ratio'] = SupplyBalanceDF['X']/SupplyBalanceDF['Y'] 
SupplyBalanceDF['C_ratio'] = SupplyBalanceDF['C']/SupplyBalanceDF['Y'] 
SupplyBalanceDF['G_ratio'] = SupplyBalanceDF['G']/SupplyBalanceDF['Y']
SupplyBalanceDF['I_ratio'] = SupplyBalanceDF['I']/SupplyBalanceDF['Y'] 

## Capital, employment and population

**Employment and population:**

In [9]:
display(EmploymentPopuplation.variable_levels('SOCIO', language='en'))

,id,text
0,EMPM_DC,Employment (number)
1,EMPM_NC,"Employment, national concept"
2,POP,Population


**Define variables of interest:**

In [10]:
params = {
    'table': 'NAHB',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'SOCIO', 'values': ['EMPM_DC','POP']},
        {'code': 'Tid', 'values': [f'>={FirstYear}<']}
        ]
    }
EmploymentPopuplationVar = EmploymentPopuplation.get_data(params=params).sort_values(['TID'])

**Capital:**

**Define variables of interest:**

In [36]:
params = {
    'table': 'NAHD34',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'TRANSAKT', 'values': ['B8GK']},
        {'code': 'Tid', 'values': [f'>={FirstYear}<']}
        ]
    }
CapitalVar = Capital.get_data(params=params).sort_values(['TID'])


**Create dataframe:**

In [37]:
CapitalEmploymentPopuplationDF = pd.DataFrame()
CapitalEmploymentPopuplationDF['L'] = EmploymentPopuplationVar[['TID','INDHOLD']].loc[EmploymentPopuplationVar['SOCIO'] == 'Employment (number)'].copy().set_index(['TID'])
CapitalEmploymentPopuplationDF['N'] = EmploymentPopuplationVar[['TID','INDHOLD']].loc[EmploymentPopuplationVar['SOCIO'] == 'Population'].copy().set_index(['TID'])
CapitalEmploymentPopuplationDF['K'] = CapitalVar[['TID','INDHOLD']].loc[CapitalVar['TRANSAKT'] == 'B.8g Saving, gross'].copy().set_index(['TID'])
CapitalEmploymentPopuplationDF = CapitalEmploymentPopuplationDF.apply(pd.to_numeric)
CapitalEmploymentPopuplationDF['L_ratio'] = CapitalEmploymentPopuplationDF['L']/CapitalEmploymentPopuplationDF['N'] 

## Unemployment rate

**Define variables of interest:**

In [39]:
params = {
    'table': 'AULAAR',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'KØN', 'values': ['TOT']},
        {'code': 'PERPCT', 'values': ['L10']},
        {'code': 'Tid', 'values':[f'>={FirstYear}<']}
        ]
    }
UnemploymentVar = Unemployment.get_data(params=params).set_index(['TID']).sort_values(['TID'])

**Create dataframe:**

In [40]:
UnemploymentDF = pd.DataFrame()
UnemploymentDF['U'] = UnemploymentVar[['INDHOLD']].loc[UnemploymentVar['PERPCT'] == 'Per cent of the labour force'].copy()
UnemploymentDF = UnemploymentDF.apply(pd.to_numeric)

## Income and wealth lifecycles

**Income:**

In [48]:
Income.tablesummary(language='en')

Table INDKP201: Main table for personal income statistics by type of income, sex, age, population, price unit, unit and time
Last update: 2022-11-24T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,INDKOMSTTYPE,39,100,1 Disposable income (2+30-31-32-35),290,Taxable income,False
1,KOEN,3,MOK,"Men and women, total",K,Women,False
2,ALDER,15,14TOT,"Total, 15 years and over",80-00,80 years and over,False
3,POPU,2,5000,All regardless of having the type of income,5020,Only people with the type of income,False
4,PRISENHED,2,005,Constant prices,006,Nominal prices,False
5,ENHED,5,0050,Mean (DKK),105,People (Number),False
6,Tid,26,1996,1996,2021,2021,True


**Define variables of interest:**

In [53]:
params = {
    'table': 'INDKP201',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'INDKOMSTTYPE', 'values': ['100','105']},
        {'code': 'KOEN', 'values': ['MOK']},
        {'code': 'ALDER', 'values': ['*']},	
        {'code': 'POPU', 'values': ['5000']},	
        {'code': 'PRISENHED', 'values': ['005']},
        {'code': 'ENHED', 'values': ['0050']},
        {'code': 'Tid', 'values':[f'>={FirstYear}<']}
        ]
    }
IncomeVar = Income.get_data(params=params).set_index(['TID']).sort_values(['TID'])
IncomeVar

,INDKOMSTTYPE,KOEN,ALDER,POPU,PRISENHED,ENHED,INDHOLD
TID,,,,,,,
2000,1 Disposable income (2+30-31-32-35),"Men and women, total","Total, 15 years and over",All regardless of having the type of income,Constant prices,Mean (DKK),189300
2000,"2 Pre-tax Income, total (3+7+22+26+29)","Men and women, total",80 years and over,All regardless of having the type of income,Constant prices,Mean (DKK),197300
2000,"2 Pre-tax Income, total (3+7+22+26+29)","Men and women, total",75-79 years,All regardless of having the type of income,Constant prices,Mean (DKK),202700
2000,"2 Pre-tax Income, total (3+7+22+26+29)","Men and women, total",70-74 years,All regardless of having the type of income,Constant prices,Mean (DKK),208400
2000,"2 Pre-tax Income, total (3+7+22+26+29)","Men and women, total",65-69 years,All regardless of having the type of income,Constant prices,Mean (DKK),223100
...,...,...,...,...,...,...,...
2021,1 Disposable income (2+30-31-32-35),"Men and women, total",15-19 years,All regardless of having the type of income,Constant prices,Mean (DKK),36200
2021,1 Disposable income (2+30-31-32-35),"Men and women, total","Total, 15 years and over",All regardless of having the type of income,Constant prices,Mean (DKK),267400
2021,"2 Pre-tax Income, total (3+7+22+26+29)","Men and women, total",75-79 years,All regardless of having the type of income,Constant prices,Mean (DKK),290600


**Create dataframe:**

In [59]:
IncomeDF = pd.DataFrame()
IncomeDF = IncomeVar[['INDKOMSTTYPE','ALDER','INDHOLD']].copy()
IncomeDF

,INDKOMSTTYPE,ALDER,INDHOLD
TID,,,
2000,1 Disposable income (2+30-31-32-35),"Total, 15 years and over",189300
2000,"2 Pre-tax Income, total (3+7+22+26+29)",80 years and over,197300
2000,"2 Pre-tax Income, total (3+7+22+26+29)",75-79 years,202700
2000,"2 Pre-tax Income, total (3+7+22+26+29)",70-74 years,208400
2000,"2 Pre-tax Income, total (3+7+22+26+29)",65-69 years,223100
...,...,...,...
2021,1 Disposable income (2+30-31-32-35),15-19 years,36200
2021,1 Disposable income (2+30-31-32-35),"Total, 15 years and over",267400
2021,"2 Pre-tax Income, total (3+7+22+26+29)",75-79 years,290600


**Wealth**:

**Define variables of interest:**

In [60]:
Wealth.tablesummary(language='en')

Table FORMUE11: Wealth by type of wealth, unit, age, sex, population and time
Last update: 2022-12-20T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,FORM1,46,FGNF2020,"Net Wealth, (2020-definition A+B+CX-D-E-F)",FGF4,F.4. Other debt to the public sector,False
1,ENHED,9,200,Median (Constant prices 2021-level),240,Number of people in population 31.12,False
2,ALDER,16,1802,18 years and over,9099,90 years and over,False
3,KØN,3,MOK,"Men and women, total",K,Women,False
4,POPU,2,5005,Entire population,5025,Only people with the selected type of wealth,False
5,Tid,8,2014,2014,2021,2021,True


In [61]:
display(Wealth.variable_levels('ENHED', language='en'))

,id,text
0,200,Median (Constant prices 2021-level)
1,205,Lower quartile (Constant prices 2021-leel)
2,210,Upper quartile (Constant prices 2021-level)
3,215,Average (Constant prices 2021-level)
4,220,Median (Nominal prices)
5,225,Lower quartile (Nominal prices)
6,230,Upper quartile (Nominal prices)
7,235,Average (Nominal prices)
8,240,Number of people in population 31.12


In [96]:
params = {
    'table': 'FORMUE11',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'FORM1', 'values': ['FGNF2020']},
        {'code': 'ENHED', 'values': ['215']},
        {'code': 'ALDER', 'values': ['*']},	
        {'code': 'KØN', 'values': ['MOK']},
        {'code': 'POPU', 'values': ['5005']},	
        {'code': 'Tid', 'values':[f'>=2014<']}
        ]
    }
WealthVar = Wealth.get_data(params=params).set_index(['TID']).sort_values(['TID'])

In [101]:
WealthDF = pd.DataFrame()
WealthDF = WealthVar[['TID','ALDER','INDHOLD']].copy()
WealthDF

,TID,ALDER,INDHOLD
0,2014,18 years and over,..
15,2014,90 years and over,..
14,2014,85-89 years,..
13,2014,80-84 years,..
12,2014,75-79 years,..
...,...,...,...
113,2021,18-24 years,187629
112,2021,18 years and over,1938654
126,2021,85-89 years,1967944
118,2021,45-49 years,2197622


In [109]:
WealthDF2 = pd.DataFrame()
WealthDF2['18 years and over'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '18 years and over'].set_index(['TID'])
WealthDF2['18-24 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '18-24 years'].set_index(['TID'])
WealthDF2['25-29 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '25-29 years'].set_index(['TID'])
WealthDF2['30-34 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '30-34 years'].set_index(['TID'])
WealthDF2['35-39 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '35-39 years'].set_index(['TID'])
WealthDF2['40-44 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '40-44 years'].set_index(['TID'])
WealthDF2['45-49 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '45-49 years'].set_index(['TID'])
WealthDF2['50-54 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '50-54 years'].set_index(['TID'])
WealthDF2['55-59 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '55-59 years'].set_index(['TID'])
WealthDF2['60-64 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '60-64 years'].set_index(['TID'])
WealthDF2['65-69 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '65-69 years'].set_index(['TID'])
WealthDF2['70-74 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '70-74 years'].set_index(['TID'])
WealthDF2['75-79 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '75-79 years'].set_index(['TID'])
WealthDF2['80-84 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '80-84 years'].set_index(['TID'])
WealthDF2['85-89 years'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '85-89 years'].set_index(['TID'])
WealthDF2['90 years and over'] = WealthDF[['TID','INDHOLD']].loc[WealthVar['ALDER'] == '90 years and over'].set_index(['TID'])
WealthDF2

,18 years and over,18-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85-89 years,90 years and over
TID,,,,,,,,,,,,,,,,
2014,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2015,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2016,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2017,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2018,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2019,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2020,1694985,137927,226799,437082,806615,1282259,1890615,2300089,2677850,3061997,3057230,2878569,2605665,2059096,1755330,1609966
2021,1938654,187629,295654,518780,948522,1482725,2197622,2718674,3069287,3426038,3402934,3173698,2964666,2370539,1967944,1761446


## Steady state values

In [41]:
frames = [SupplyBalanceDF, CapitalEmploymentPopuplationDF, UnemploymentDF]

In [42]:
All_variables =  pd.concat(frames, axis=1)
All_variables

,Y,M,X,C,G,I,M_ratio,X_ratio,C_ratio,G_ratio,I_ratio,L,N,K,L_ratio,U
TID,,,,,,,,,,,,,,,,
2000,1677.2,548.7,696.9,752.6,412.2,348.2,0.327152,0.415514,0.448724,0.245767,0.207608,2755149.0,5338491.0,318423.0,0.516091,4.9
2001,1691.0,561.9,720.3,754.3,420.0,343.0,0.332289,0.425961,0.446067,0.248374,0.202839,2782473.0,5357285.0,343436.0,0.519381,4.7
2002,1698.9,597.7,751.7,765.2,429.0,338.9,0.351816,0.442463,0.450409,0.252516,0.199482,2784200.0,5375575.0,343526.0,0.517935,4.8
2003,1705.5,591.6,742.7,775.4,430.0,336.6,0.346878,0.435473,0.454647,0.252125,0.197361,2758653.0,5389521.0,351281.0,0.511855,5.8
2004,1751.0,633.8,765.1,811.5,436.6,359.4,0.361965,0.436950,0.463449,0.249343,0.205254,2744153.0,5403040.0,374174.0,0.507891,5.8
2005,1792.0,705.3,824.2,841.5,441.8,379.3,0.393583,0.459933,0.469587,0.246540,0.211663,2783163.0,5418559.0,418680.0,0.513635,5.1
2006,1862.1,803.9,909.4,866.3,453.1,427.7,0.431717,0.488373,0.465227,0.243327,0.229687,2845937.0,5436756.0,464694.0,0.523462,3.9
2007,1879.0,850.8,942.6,881.6,458.5,439.1,0.452794,0.501650,0.469186,0.244013,0.233688,2912410.0,5460399.0,464702.0,0.533369,2.7
2008,1869.4,891.4,979.1,885.9,473.4,418.0,0.476837,0.523751,0.473895,0.253236,0.223601,2946869.0,5492726.0,484618.0,0.536504,1.9


In [43]:
All_variables.mean()

Y          1.897430e+03
M          8.675261e+02
X          9.948696e+02
C          8.806391e+02
G          4.822435e+02
I          4.021348e+02
M_ratio    4.520847e-01
X_ratio    5.197136e-01
C_ratio    4.641320e-01
G_ratio    2.545011e-01
I_ratio    2.109130e-01
L          2.848983e+06
N          5.574142e+06
K          5.100308e+05
L_ratio    5.112114e-01
U          4.004545e+00
dtype: float64